# Trabalho da 'turma de DM' - Data Mining do curso 'BI MASTER 2020-2'
Estudo de caso: Previsão de vendas de produtos específico em uma empresa varejista

In [ ]:
import datetime
import math
import os

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
# chamamos o método 'set' do seaborn para ajustar os valores padrão de exibição dos gráficos
sns.set_theme()
sns.set()

In [ ]:
# variáveis globais "constantes" apontando para os conjuntos de dados
PATH_DATASET="base"
PATH_DATASET_PRODUTO=os.path.join(PATH_DATASET,"BaseProduto.csv")
PATH_DATASET_VENDA=os.path.join(PATH_DATASET,"BaseVenda.csv")
PATH_DATASET_LOJA=os.path.join(PATH_DATASET,"BaseLoja.csv")

## Carga dos conjuntos de dados: 'vendas', 'lojas' e 'produtos'

In [ ]:
%time df_venda = pd.read_csv(PATH_DATASET_VENDA, delimiter='|', encoding='cp1252')

In [ ]:
%time df_loja = pd.read_csv(PATH_DATASET_LOJA, delimiter='|', encoding='cp1252')

In [ ]:
%time df_produto = pd.read_csv(PATH_DATASET_PRODUTO, delimiter='|', encoding='cp1252', low_memory=False)

In [ ]:
df_produto.head()

##### Filtramos o conjunto de dados para obter apenas calçados femininos vendidos no Rio de Janeiro


In [ ]:
df_venda_preparado = df_venda[(df_venda.SECAO == 'CALCADOS') & 
                              (df_venda.GRUPO == 'FEMININO') &
                              (df_venda.CIDADE == 'RIO DE JANEIRO')]

In [ ]:
print(df_venda.shape)
print(df_venda_preparado.shape)

In [ ]:
perc_dados_selec_orig = (df_venda_preparado.shape[0] / df_venda.shape[0]) * 100
print(f'Percentual de dados selecionados: {perc_dados_selec_orig:.2f} %')

##### Removemos colunas desnecessárias ao processamento

In [ ]:
df_venda_preparado = df_venda_preparado.drop(['NumeroBoleta', 
                                              'Loja', 
                                              'UF', 
                                              'CIDADE',
                                              'BAIRRO', 
                                              'Produto_Codigo', 
                                              'SECAO', 
                                              'GRUPO', 
                                              'CATEGORIA', 
                                              'COR', 
                                              'TAMANHO', 
                                              'PrecoVenda', 
                                              'PrecoTransacao'], axis=1)

In [ ]:
# verificamos se existem valores nulos nos dados
df_venda_preparado.isna().sum()

In [ ]:
# número de registros antes da aglutinação
qtd_dados_venda_preparado_antes = df_venda_preparado.shape[0]

# aglutinamos (somando) os dados que ocorrem na mesma CriacaoReferencia
df_venda_preparado = df_venda_preparado.groupby(by=['CriacaoReferencia'], as_index=False)['Quantidade'].sum()

# número de registros depois da aglutinação
qtd_dados_venda_preparado_depois = df_venda_preparado.shape[0]

In [ ]:
df_venda_preparado.shape

In [ ]:
perc_reducao_dados_aglutinados = (1 - (qtd_dados_venda_preparado_depois / qtd_dados_venda_preparado_antes)) * 100
print(f'Percentual de redução do número de registros via aglutinação: {perc_reducao_dados_aglutinados:.2f} %')

In [ ]:
# formatos padronizados para as datas
fmt_dt_iso='%Y-%m-%d'
fmt_dt_ano_mes='%Y-%m'
fmt_dt_ano_mes_semana='%Y-%m-%W'

In [ ]:
# funções auxiliares para calcular as janelas de tempo
f_data_menos_uma_semana   = lambda d: d + relativedelta(weeks=-1)
f_data_menos_duas_semanas = lambda d: d + relativedelta(weeks=-2)
f_data_menos_um_ano       = lambda d: d + relativedelta(years=-1)

def formatar_data(df, coluna, formato):
    """Aplica um formato de data em uma coluna de um dataframe e a retorna."""
    return df[coluna].dt.strftime(formato)

def aplicar_funcoes(df, colunas_e_funcoes):
    """Percorre as tuplas de 'colunas_e_funcoes' e, para cada dicionario nela contidos, faz:
    - aplica a 'funcao' na coluna 'col_org', gravando o resultado na coluna 'col_dst' do dataframe 'df'
    """
    for e in colunas_e_funcoes:
        col_dst = e['col_dst']
        col_org = e['col_org']
        f_offset_data = e['funcao']
                
        # aplica a funcao 'f_offset_data' na coluna 'col_org' e 
        # salva o resultado na coluna 'col_dst'
        df[col_dst] = df[col_org].map(f_offset_data)
    
def aplicar_formatos_data(df, colunas_e_formatos):
    """Percorre as tuplas de 'colunas_e_formatos' e, para cada dicionario nela contidos, faz:
    - aplica o 'fmt_data' na coluna 'col_org', gravando o resultado na coluna 'col_dst' do dataframe 'df'
    """
    for e in colunas_e_formatos:
        col_dst = e['col_dst']
        col_org = e['col_org']
        fmt_data = e['fmt_data']
        
        # aplica o formato de data 'fmt_data' na coluna 'col_org' e 
        # salva o resultado na coluna 'col_dst'
        df[col_dst] = formatar_data(df, col_org, fmt_data)

In [ ]:
# criamos uma coluna temporária de data da venda no padrão ISO
%time df_venda_preparado['DataVenda'] = pd.to_datetime(df_venda_preparado['CriacaoReferencia'],format=fmt_dt_iso)

In [ ]:
# criamos uma coluna de ano-mês para facilitar as aglutinações dos dados
%time df_venda_preparado['AnoMes'] =  formatar_data(df_venda_preparado, 'DataVenda', fmt_dt_ano_mes)

In [ ]:
# criamos colunas temporárias no dataframe para armazenar as datas de interesse
# essas colunas serão formatadas na sequência

# col_dst recebe funcao aplicada em col_org, na ordem das tuplas
colunas_e_offsets_de_tempo = (
    {'col_dst':'Data1SemanaAnterior',    'col_org': 'DataVenda',       'funcao': f_data_menos_uma_semana},
    {'col_dst':'Data2SemanaAnterior',    'col_org': 'DataVenda',       'funcao': f_data_menos_duas_semanas},
    {'col_dst':'DataAnoAnterior',        'col_org': 'DataVenda',       'funcao': f_data_menos_um_ano},
    {'col_dst':'DataAnoAnterior1Semana', 'col_org': 'DataAnoAnterior', 'funcao': f_data_menos_uma_semana},
    {'col_dst':'DataAnoAnterior2Semana', 'col_org': 'DataAnoAnterior', 'funcao': f_data_menos_duas_semanas}
)

# aplicamos as funções, medindo o tempo (demora bastante)
%time aplicar_funcoes(df_venda_preparado, colunas_e_offsets_de_tempo)

In [ ]:
df_venda_preparado.shape

In [ ]:
# criamos colunas de datas específicas (formatadas) no dataframe para auxiliar a geração das janelas de
# tempo para o processamento dos algoritmos de regressão

# col_dst recebe formato de data aplicado em col_org, na ordem das tuplas
colunas_e_formatos = (
    {'col_dst':'AnoMesSemana',                   'col_org': 'DataVenda',              'fmt_data': fmt_dt_ano_mes_semana},
    {'col_dst':'AnoMesSemana1SemanaAnterior',    'col_org': 'Data1SemanaAnterior',    'fmt_data': fmt_dt_ano_mes_semana},
    {'col_dst':'AnoMesSemana2SemanaAnterior',    'col_org': 'Data2SemanaAnterior',    'fmt_data': fmt_dt_ano_mes_semana},
    {'col_dst':'AnoMesSemanaAnoAnterior',        'col_org': 'DataAnoAnterior',        'fmt_data': fmt_dt_ano_mes_semana},
    {'col_dst':'AnoMesSemanaAnoAnterior1Semana', 'col_org': 'DataAnoAnterior1Semana', 'fmt_data': fmt_dt_ano_mes_semana},
    {'col_dst':'AnoMesSemanaAnoAnterior2Semana', 'col_org': 'DataAnoAnterior2Semana', 'fmt_data': fmt_dt_ano_mes_semana}
)

# aplicamos os formatos, medindo o tempo (demora bastante)
%time aplicar_formatos_data(df_venda_preparado, colunas_e_formatos)

In [ ]:
df_venda_preparado.shape

In [ ]:
# removemos, do dataframe, as colunas que não são mais necessárias para processamento posterior
colunas_a_remover = ['CriacaoReferencia',
                     'DataVenda',
                     'Data1SemanaAnterior',
                     'Data2SemanaAnterior',
                     'DataAnoAnterior',
                     'DataAnoAnterior1Semana',
                     'DataAnoAnterior2Semana']

%time df_venda_preparado.drop(colunas_a_remover, axis=1, inplace=True)

In [ ]:
df_venda_preparado.shape

---

### Análise Estatística

#### Grafico da série temporal e da Autocorrelação - Mensal

In [ ]:
df_venda_agrupado_mes = df_venda_preparado.groupby(by=['AnoMes'], as_index=False)['Quantidade'].sum()
df_venda_agrupado_mes = df_venda_agrupado_mes.sort_values(['AnoMes'])

In [ ]:
green = sns.color_palette("deep",8)[2]
blue = sns.color_palette("deep",8)[0]

fig, ax = plt.subplots(figsize=(20,5))
df_venda_agrupado_mes.plot(x="AnoMes",y="Quantidade",color="g", fontsize=15, ax=ax)
plt.xlabel("Ano/Mês",fontsize=15)
plt.title("Quantidade de Itens Vendidos por Mês/Ano", fontsize=15)
plt.ylabel("Quantidade Vendida", fontsize=15)
plt.show()


fig, ax = plt.subplots(figsize=(20,5))
plot_acf(df_venda_agrupado_mes.Quantidade, ax=ax)
plt.title("Auto Correlação", fontsize=15)
plt.xlabel("Lag",fontsize=15)
plt.ylabel("Correlação", fontsize=15)
plt.show()

#### Grafico da série temporal e da Autocorrelação - Semanal

In [ ]:
df_venda_agrupado_semana = df_venda_preparado.groupby(by=['AnoMesSemana'], as_index=False)['Quantidade'].sum()

In [ ]:
green = sns.color_palette("deep",8)[2]
blue = sns.color_palette("deep",8)[0]

fig, ax = plt.subplots(figsize=(20,5))
df_venda_agrupado_semana.plot(x="AnoMesSemana",y="Quantidade",color="g", fontsize=15, ax=ax)
plt.xlabel("Ano/Mês/Semana",fontsize=15)
plt.title("Quantidade de Itens Vendidos por Mês/Ano/Semana", fontsize=15)
plt.ylabel("Quantidade Vendida", fontsize=15)
plt.show()


fig, ax = plt.subplots(figsize=(20,5))
plot_acf(df_venda_agrupado_semana.Quantidade, ax=ax)
plt.title("Auto Correlação", fontsize=15)
plt.xlabel("Lag",fontsize=15)
plt.ylabel("Correlação", fontsize=15)
plt.show()

### Previsões Estatísticas

#### Média Móvel

In [ ]:
def plot_moving_average(series, window, plot_intervals=False, scale=1.96):

    rolling_mean = series.rolling(window=window).mean()
    
    plt.figure(figsize=(17,8))
    plt.title('Moving average\n window size = {}'.format(window))
    plt.plot(rolling_mean, 'g', label='Rolling mean trend')
    
    #Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bound = rolling_mean - (mae + scale * deviation)
        upper_bound = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bound, 'r--', label='Upper bound / Lower bound')
        plt.plot(lower_bound, 'r--')
            
    plt.plot(series[window:], label='Actual values')
    plt.legend(loc='best')
    plt.grid(True)
    

#### Média Móvel com janelas para agrupamento mês

In [ ]:
plot_moving_average(df_venda_agrupado_mes.Quantidade, 5)
plot_moving_average(df_venda_agrupado_mes.Quantidade, 10)
plot_moving_average(df_venda_agrupado_mes.Quantidade, 12, plot_intervals=True)

#### Média Móvel com janelas para agrupamento Semanal

In [ ]:
plot_moving_average(df_venda_agrupado_semana.Quantidade, 2)
plot_moving_average(df_venda_agrupado_semana.Quantidade, 5)
plot_moving_average(df_venda_agrupado_mes.Quantidade, 11, plot_intervals=True)

### Amortecimento Exponencial

In [ ]:
def exponential_smoothing(series, alpha):

    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result
  
def plot_exponential_smoothing(series, alphas):
 
    plt.figure(figsize=(17, 8))
    for alpha in alphas:
        plt.plot(exponential_smoothing(series, alpha), label="Alpha {}".format(alpha))
    plt.plot(series.values, "c", label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Exponential Smoothing")
    plt.grid(True);



#### Mensal

In [ ]:
plot_exponential_smoothing(df_venda_agrupado_mes.Quantidade, [0.05, 0.3])

#### Semanal

In [ ]:
plot_exponential_smoothing(df_venda_agrupado_semana.Quantidade, [0.05, 0.3])

### ARIMA

In [ ]:
X = df_venda_agrupado_semana["Quantidade"].values

split = int(0.8*len(X))
train, test = X[0:split], X[split:]

history = [x for x in train]
predictions = []
for t in range(len(test)):
	model = ARIMA(history, order=(5,1,0))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
    
	obs = test[t]
	history.append(obs)
mse = mean_squared_error(test, predictions)

print(f"MSE error: {mse}")

plt.figure(figsize=(17,8))
plt.plot(test)
plt.plot(predictions, color='red')
plt.title("ARIMA fit to Sales Data",fontsize=15)
plt.xticks([])
plt.show()

## Previsões Machine Learning 

#### Todas as vendas realizadas agrupadas por semana

In [ ]:
df_venda_historico_agrupado = df_venda_preparado.groupby(by=['AnoMesSemana'], as_index=False)['Quantidade'].sum()

#### Métricas de Erro

In [ ]:
def calcula_metricas_erro(y_pred,y_test, number_features):
    rmse = math.sqrt(mean_squared_error(y_pred,y_test))
    print('RMSE: ', rmse)
    
    mse = mean_squared_error(y_pred,y_test)
    print('MSE: ',mse)    
    
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print('MAPE: ',mape, '%')
    
    r2 = r2_score(y_pred,y_test)
    print('R2 Score: ', r2)
    
    if (number_features is not None and number_features > 0): 
        number_samples = len(y_pred)
        adj_r2_score = 1-(1-r2)*(number_samples-1)/(number_samples-number_features-1)
        print('R2 Ajustado: ', adj_r2_score)
    

In [ ]:
def mostra_grafico_previsao(y_pred, y_test):
    plt.plot(y_pred, label='previsto', marker='o')
    plt.plot(y_test, label='real', marker='+')
    plt.ylabel("Venda")
    plt.title("Previsão x Vendas")
    plt.legend()
    plt.show()

#### Montagem dos diferentes datasets para treino 

##### Dados da série histórica com a janela informada

In [ ]:
def monta_dataset_por_janela(dataset=None, window=12):
    dataSize = len(dataset)
    X = []
    y = []
    for i in range(window, dataSize):
        X.append(dataset.iloc[i-window:i, 1])
        y.append(dataset.iloc[i, 1])
    X, y = np.array(X), np.array(y)
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    return x_train, x_test, y_train, y_test
    

##### Penúltima e antepenúltima semana/ mesma semana do ano anterior / penúltima e antepenúltima semana do ano anterior.

In [ ]:
def monta_dataset_ano_anterior(dataset):
    df_venda_ml = dataset[dataset.AnoMesSemana >= '2020-01-02'].sort_values(['AnoMesSemana'])
    df_venda_agrupado_ml = df_venda_ml.groupby('AnoMesSemana', as_index=False).agg({
        'Quantidade':'sum',
        'AnoMesSemana1SemanaAnterior': 'max',
        'AnoMesSemana2SemanaAnterior': 'max',
        'AnoMesSemanaAnoAnterior': 'max',
        'AnoMesSemanaAnoAnterior1Semana':'max',
        'AnoMesSemanaAnoAnterior2Semana':'max'})
    
    df_venda_ml_final = pd.merge(df_venda_agrupado_ml, df_venda_historico_agrupado, left_on="AnoMesSemanaAnoAnterior", right_on="AnoMesSemana", suffixes=("","_AnoAnterior"))

    df_venda_ml_final = pd.merge(df_venda_ml_final, df_venda_historico_agrupado, left_on="AnoMesSemanaAnoAnterior1Semana", right_on="AnoMesSemana", suffixes=("","_AnoAnterior1Semana"))
    df_venda_ml_final = pd.merge(df_venda_ml_final, df_venda_historico_agrupado, left_on="AnoMesSemanaAnoAnterior2Semana", right_on="AnoMesSemana", suffixes=("","_AnoAnterior2Semana"))
    df_venda_ml_final = pd.merge(df_venda_ml_final, df_venda_historico_agrupado, left_on="AnoMesSemana1SemanaAnterior", right_on="AnoMesSemana", suffixes=("","_1SemanaAnterior"))
    df_venda_ml_final = pd.merge(df_venda_ml_final, df_venda_historico_agrupado, left_on="AnoMesSemana2SemanaAnterior", right_on="AnoMesSemana", suffixes=("","_2SemanaAnterior"))
    
    df_venda_ml_final = df_venda_ml_final[['AnoMesSemana','Quantidade_1SemanaAnterior','Quantidade_2SemanaAnterior','Quantidade_AnoAnterior','Quantidade_AnoAnterior1Semana','Quantidade_AnoAnterior2Semana','Quantidade']]
    
    np_dataset =  df_venda_ml_final.to_numpy()
    X = np_dataset[:,1:-1]
    y = np_dataset[:,-1]
    
    np.random.seed(0)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    return x_train, x_test, y_train, y_test

##### Algoritmos de Machine Learning

In [ ]:
def train_predict_with_model(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    calcula_metricas_erro(y_pred=y_pred, y_test=y_test, number_features=X_test.shape[1])
    mostra_grafico_previsao(y_pred=y_pred, y_test=y_test)
    
    # imprimimos os parametros usados no modelo, caso tenha sido usado o GridSearchCV
    # para determinar um modelo.
    try:
        print(model.best_params_)
    except AttributeError:
        pass

In [ ]:
def create_tuned_random_forest_regressor():
    # veja os parâmetros em:
    # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
    tuned_parameters = {
        'n_estimators': [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        'criterion': ['mse', 'mae'],
        'max_depth': [None, 1, 2, 4, 6, 8],
        'min_samples_leaf': [x for x in range(1,12,2)], # de 1 a 11 de 2 em 2
        'random_state': [0], # para facilitar a reprodução dos resultados observados
        'n_jobs': [-1] # usamos todos os processadores lógicos disponíveis no treino e na predição do modelo
    }
    
    # com o RandomForestRegressor, não podemos usar o scoring do GridSearchCV
    return GridSearchCV(RandomForestRegressor(), tuned_parameters, verbose=1)

In [ ]:
def train_predict_random_forest(X_train, X_test, y_train, y_test, best_model=True):
    if best_model:
        regressor = create_tuned_random_forest_regressor()
    else:
        regressor = RandomForestRegressor(n_estimators = 10, random_state=0)

    train_predict_with_model(X_train, X_test, y_train, y_test, regressor)

In [ ]:
def create_tuned_svm():
    # veja os parâmetros em:
    # https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
    tuned_parameters = {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
        'tol': [1e-2, 1e-3, 1e-4, 1e-5],
        'C': list(np.arange(1.0, 16.0, 2.0)), # C de 1 a 16 de 2 em 2
        'epsilon': [0.1, 1e-2, 1e-3]
    }
    
    return GridSearchCV(svm.SVR(), tuned_parameters, verbose=1)    

In [ ]:
def train_predict_svm(X_train, X_test, y_train, y_test, best_model=True):
    # veja os parâmetros em:
    # https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
    if best_model:
        regressor = create_tuned_svm()
    else:
        regressor = svm.SVR(kernel='linear', C=15.0)

    train_predict_with_model(X_train, X_test, y_train, y_test, regressor)
    

---

### Treino e Teste dos Modelos de ML

In [ ]:
# funções geradoras dos conjuntos de dados de cada cenário que iremos analisar
f_gerar_ano_anterior_e_corrente = lambda: monta_dataset_ano_anterior(dataset=df_venda_preparado)
f_gerar_4_semanas               = lambda: monta_dataset_por_janela(dataset=df_venda_historico_agrupado, window=4)
f_gerar_12_semanas              = lambda: monta_dataset_por_janela(dataset=df_venda_historico_agrupado, window=12)
f_gerar_24_semanas              = lambda: monta_dataset_por_janela(dataset=df_venda_historico_agrupado, window=24)
f_gerar_36_semanas              = lambda: monta_dataset_por_janela(dataset=df_venda_historico_agrupado, window=36)
f_gerar_52_semanas              = lambda: monta_dataset_por_janela(dataset=df_venda_historico_agrupado, window=52)

# um iterável ordenado de dicionários, contendo os nomes dos cenários e um lambda para gerar
# o conjunto de dados (evitamos incluir os dados no dicionário para não aumentar o footprint 
# de memória)
cenarios_analise_ml = (
    { 'nome':'ano anterior e corrente', 'f_gerar_dataset': f_gerar_ano_anterior_e_corrente},
    { 'nome':'janela de 4 semanas',     'f_gerar_dataset': f_gerar_4_semanas},
    { 'nome':'janela de 12 semanas',    'f_gerar_dataset': f_gerar_12_semanas},
    { 'nome':'janela de 24 semanas',    'f_gerar_dataset': f_gerar_24_semanas},
    { 'nome':'janela de 36 semanas',    'f_gerar_dataset': f_gerar_36_semanas},
    { 'nome':'janela de 52 semanas',    'f_gerar_dataset': f_gerar_52_semanas}
)

algoritmos_ml = (
    { 'nome': 'Random Forest', 'f_train_predict': train_predict_random_forest},
    { 'nome': 'SVM'          , 'f_train_predict': train_predict_svm}
)

# percorremos os cenários, treinando os algoritmos de IA e fazendo previsões nos dados
# cenários x algorítmos (com e sem otimização de hiper-parâmetros)
for idx, cenario in enumerate(cenarios_analise_ml):
    nome_cenario = cenario['nome']
    f_gerar_dataset = cenario['f_gerar_dataset']
    X_train, X_test, y_train, y_test = f_gerar_dataset() # aqui, chamamos a função lambda para obter os dados do cenário
    
    print(f'Cenário {idx + 1}: {nome_cenario}' + os.linesep)
    
    for alg in algoritmos_ml:
        nome_alg = alg['nome']
        f_train_predict = alg['f_train_predict']
        
        for best_model in (False, True):
            s_best_model = 'melhores' if best_model else 'padrão'
            
            print(f'Algorítmo: {nome_alg}')
            print(f'Parâmetros do modelo: {s_best_model}.' + os.linesep)
            
            %time f_train_predict(X_train, X_test, y_train, y_test, best_model)
            print()
    
    print(('-' * 50) + os.linesep)

---